In [41]:
import requests
from bs4 import BeautifulSoup 
import chardet

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Accept-Language': 'en-US,en;q=0.9',
    'Referer': 'http://example.com'
}

url = "https://serieson.naver.com/v3/movie/ranking/realtime"

response=requests.get(url, headers=headers)
if response.status_code==200:
    encoding=chardet.detect(response.content)['encoding']
    soup = BeautifulSoup(response.text,'html.parser')  
else:
    print("HTTP request failed")

titles=soup.select('span.Title_title__s9o0D')
types=soup.select('span.Price_text__pRk_f')
prices=soup.select('span.Price_price__GqXqo')
title_li=[]; type_li=[]; price_li=[];
for i in range(len(titles)):
    title_li.append(titles[i].text)
    type_li.append(types[i].text)
    price_li.append(prices[i].text.replace('캐시',''))
df=pd.DataFrame({'Titles':title_li, 'Types':type_li, 'Prices':price_li})
df.head(100)

## title
#container > div.RankingPage_ranking_wrap__GB855 > ol > li:nth-child(1) > a > div.Poster_info_area__HGRo_.Poster_margin_left__sLY0L > span
## Type
#container > div.RankingPage_ranking_wrap__GB855 > ol > li:nth-child(1) > a > div.Poster_info_area__HGRo_.Poster_margin_left__sLY0L > div > div > div > div:nth-child(1) > span.Price_text__pRk_f
## Price
#container > div.RankingPage_ranking_wrap__GB855 > ol > li:nth-child(1) > a > div.Poster_info_area__HGRo_.Poster_margin_left__sLY0L > div > div > div > div:nth-child(1) > span.Price_price__GqXqo

,Titles,Types,Prices
0,인사이드 아웃(패키지상품 : 더빙판 + 부가영상 추가증정),구매,"7,150"
1,파묘,대여,"7,000"
2,명탐정 코난 VS 괴도 키드,대여,"11,000"
3,설계자,대여,"11,000"
4,랜드 오브 배드,대여,"5,000"
...,...,...,...
95,컨저링,대여,"1,300"
96,아티스트 봉만대_무삭제판,구매,"6,500"
97,닥터(무삭제 감독판),구매,"1,000"
98,크루즈 패밀리(패키지: 자막판+더빙판),구매,"5,000"


In [30]:
#### Other Option using Copy/copy.selector

#container > div.RankingPage_ranking_wrap__GB855 > ol > li:nth-child(2) > a > div.RankingNumber_ranking_area__p8B_q > em > span
## title
#container > div.RankingPage_ranking_wrap__GB855 > ol > li:nth-child(1) > a > div.Poster_info_area__HGRo_.Poster_margin_left__sLY0L > span
## Type
#container > div.RankingPage_ranking_wrap__GB855 > ol > li:nth-child(1) > a > div.Poster_info_area__HGRo_.Poster_margin_left__sLY0L > div > div > div > div:nth-child(1) > span.Price_text__pRk_f
## Price
#container > div.RankingPage_ranking_wrap__GB855 > ol > li:nth-child(1) > a > div.Poster_info_area__HGRo_.Poster_margin_left__sLY0L > div > div > div > div:nth-child(1) > span.Price_price__GqXqo

titles = []
types = []
prices = []
movie_tags = soup.select('div > ol > li')
for tag in movie_tags[:100]:
    title_tag = tag.select_one('a > div > span')
    if title_tag:
        titles.append(title_tag.text.strip())
    else:
        titles.append(None)
    type_tag = tag.select_one('a > div > div > div > div > div > span.Price_text__pRk_f')
    if type_tag:
        types.append(type_tag.text.strip())
    else:
        types.append(None)

    price_tag = tag.select_one('a > div > div > div > div > div > span.Price_price__GqXqo')
    if price_tag:
        prices.append(price_tag.text.strip().replace('캐시',''))
    else:
        prices.append(None)

df = pd.DataFrame({
    'Title': titles,
    'Type': types,
    'Price': prices
})
df

,Title,Type,Price
0,인사이드 아웃(패키지상품 : 더빙판 + 부가영상 추가증정),구매,"7,150"
1,상승23,대여,"7,000"
2,상승1,대여,"11,000"
3,상승13,대여,"11,000"
4,랜드 오브 배드,대여,"5,000"
...,...,...,...
95,컨저링,대여,"1,300"
96,아티스트 봉만대_무삭제판,구매,"6,500"
97,닥터(무삭제 감독판),구매,"1,000"
98,크루즈 패밀리(패키지: 자막판+더빙판),구매,"5,000"


Task2_0620. url = 'https://news.daum.net/politics#1'은 정치기사 1페이지인데 10페이지에 있는 기사를 모두 출력
"https://news.daum.net/politics#1"
(request랑 BS만 selenium 쓰지말고)

In [69]:
## 동적 컨텐츠 로딩: selenium 사용
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time

options = Options()
options.add_argument("--start-maximized")
options.add_experimental_option("detach", True)

driver=webdriver.Chrome(options=options)

url = 'https://news.daum.net/politics#10'
driver.get(url)
time.sleep(3)

try:
    newsstand_link=driver.find_elements(By.CSS_SELECTOR, '#timeline > ul > li:nth-child(1) > strong > a')
    print(newsstand_link.text)
except:
    print("뉴스 스탠드 링크를 찾을 수 없습니다.")

driver.quit()

뉴스 스탠드 링크를 찾을 수 없습니다.


In [52]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Accept-Language': 'en-US,en;q=0.9',
    'Referer': 'http://example.com'
}

url = "https://news.daum.net/politics#1"

response=requests.get(url, headers=headers)
if response.status_code==200:
    encoding=chardet.detect(response.content)['encoding']
    soup = BeautifulSoup(response.text,'html.parser')  
else:
    print("HTTP request failed")

links=soup.select('a.link_txt')
articles=[line.text.strip() for line in links]
df=pd.DataFrame({"Articles":articles})
#df=pd.DataFrame({"Articles":links})
df.head(len(df))


,Articles
0,‘이재명 추가 기소’에 “모든 게 李 탓? vs “대표 아니면 벌써 기소”[중립기어]
1,"여·야, 원구성 협상 또 합의점 못 찾아… 입장차만 확인"
2,핵잠·ICBM 기술 이전 길 텄나…北-러 ‘방위능력 공동조치’ 명시
3,"농해수위, 야당 주도 '양곡관리법·농안법' 상정…여당 불참"
4,"우상호 ""저도 '왕수박'으로 몰려…당에 '나 같은 역할' 안 보여"""
5,"부산시의회, 부산테크노파크 원장 '2+1년' 이후 연임 집중 질의"
6,"국힘 박상웅 국회의원, 분기별 '생활안정기금' 도입 요구"
7,"아동수당 증액법 발의 전진숙…""가족예산 아끼지 말아야""[파워초선]"
8,‘채상병 특검법’ 입법청문회 D-1…‘윗선’ 어디까지 출석할까
9,"방위사업청 ""英 방산업체 상대 6천9백억 원대 손해배상 소송 최종 승소"""


Task3_0620. 앞에서 출력한 기사 리스트를 pandas 데이터프레임으로 변환 후 csv 파일로 저장 후 다시 불러오세요.
(컬럼은 인덱스 번호화 텍스트 2개 컬럼으로 구성된 DF)

In [65]:
## inside out 제목 가져오기
import requests
import chardet
import pandas as pd

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Accept-Language': 'en-US,en;q=0.9',
    'Referer': 'http://example.com'
}

url = "https://news.daum.net/politics#1"

response=requests.get(url, headers=headers)
if response.status_code==200:
    encoding=chardet.detect(response.content)['encoding']
    soup = BeautifulSoup(response.text,'html.parser')  
else:
    print("HTTP request failed")

titles=soup.select('body > div > main > section > div > div > ul > li > strong > a')
indexes_li=[]; titles_li=[];
for i, tag in enumerate(titles, start=1):
    text=tag.get_text().strip().replace(',', '')
    matches = re.findall('[가-힣0-9]+',text)
    #print(f"{i}, {' '.join(matches)}")
    indexes_li.append(i); titles_li.append(' '.join(matches))

df_titles=pd.DataFrame({'indexes':indexes_li, 'titles':titles_li})
#df_titles.head()  
df_titles.to_csv('articles.csv', index=None)
df_reloaded=pd.read_csv('articles.csv',index_col=0)
df_reloaded.head()


,titles
indexes,
1,한동훈에 맞서 원희룡 등판 나경원 윤상현 가세해 결선 투표 갈까
2,이재명 추가 기소 에 모든 게 탓 대표 아니면 벌써 기소 중립기어
3,여 야 원구성 협상 또 합의점 못 찾아 입장차만 확인
4,핵잠 기술 이전 길 텄나 러 방위능력 공동조치 명시
5,농해수위 야당 주도 양곡관리법 농안법 상정 여당 불참
